## Kaggle Dataset의 일부를 이용한 개, 고양이 구분ㅡ

### Dog Image

In [71]:
import numpy as np
import tensorflow as tf
import os, cv2, random
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from tensorflow import keras
from keras.preprocessing import image
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
%matplotlib inline

In [46]:
seed=2020
np.random.seed(seed)
tf.random.set_seed(seed)

In [47]:
path='../dataset/animal'

In [48]:
## used for resize and in our model
ROW, COL=96, 96
dogs, cats, bears, dinos, rabbits = [],[],[],[],[]
y_dogs, y_cats, y_bears, y_dinos, y_rabbits = [],[],[],[],[]

In [49]:
dog_path=os.path.join(path, 'dog.*')
cat_path=os.path.join(path, 'cat.*')
bear_path=os.path.join(path, 'bear.*')
dino_path=os.path.join(path, 'dino.*')
rabbit_path=os.path.join(path, 'rabbit.*')

In [50]:
## Load some our dog image
for dog_img in glob(dog_path):
    dog=cv2.imread(dog_img)
    dog=cv2.cvtColor(dog,cv2.COLOR_BGR2GRAY)
    dog=cv2.resize(dog, (ROW, COL))
    dog=image.img_to_array(dog)
    dogs.append(dog)

In [51]:
## Load some our dog image
for cat_img in glob(cat_path):
    cat=cv2.imread(cat_img)
    cat=cv2.cvtColor(cat,cv2.COLOR_BGR2GRAY)
    cat=cv2.resize(cat, (ROW, COL))
    cat=image.img_to_array(cat)
    cats.append(cat)

In [52]:
## Load some our dog image
for bear_img in glob(bear_path):
    bear=cv2.imread(bear_img)
    bear=cv2.cvtColor(bear,cv2.COLOR_BGR2GRAY)
    bear=cv2.resize(bear, (ROW, COL))
    bear=image.img_to_array(bear)
    bears.append(bear)

In [53]:
## Load some our dog image
for dino_img in glob(dino_path):
    dino=cv2.imread(dino_img)
    dino=cv2.cvtColor(dino,cv2.COLOR_BGR2GRAY)
    dino=cv2.resize(dino, (ROW, COL))
    dino=image.img_to_array(dino)
    dinos.append(dino)

In [54]:
## Load some our dog image
for rabbit_img in glob(rabbit_path):
    rabbit=cv2.imread(rabbit_img)
    rabbit=cv2.cvtColor(rabbit,cv2.COLOR_BGR2GRAY)
    rabbit=cv2.resize(rabbit, (ROW, COL))
    rabbit=image.img_to_array(rabbit)
    rabbits.append(rabbit)

In [55]:
classes=['dog','cat','bear','dino','rabbit']

In [56]:
## just change the labels for 0 and 1
y_dogs=[0 for item in enumerate(dogs)]
y_cats=[1 for item in enumerate(cats)]
y_bears=[1 for item in enumerate(bears)]
y_dinos=[1 for item in enumerate(dinos)]
y_rabbits=[1 for item in enumerate(rabbits)]

In [57]:
dogs = np.asarray(dogs).astype('float32') / 255
cats = np.asarray(cats).astype('float32') / 255
bears = np.asarray(bears).astype('float32') / 255
dinos = np.asarray(dinos).astype('float32') / 255
rabbits = np.asarray(rabbits).astype('float32') / 255

y_dogs = np.asarray(y_dogs).astype('int32')
y_cats = np.asarray(y_cats).astype('int32')
y_bears = np.asarray(y_bears).astype('int32')
y_dinos = np.asarray(y_dinos).astype('int32')
y_rabbits = np.asarray(y_rabbits).astype('int32')

In [91]:
a = np.concatenate((dogs,cats), axis=0)
b = np.concatenate((a,bears), axis=0)
c = np.concatenate((b,dinos), axis=0)
x = np.concatenate((c,rabbits), axis=0)
a = np.concatenate((y_dogs, y_cats), axis=0)
b = np.concatenate((a, y_bears), axis=0)
c = np.concatenate((b, y_dinos), axis=0)
y = np.concatenate((c, y_rabbits), axis=0)
x.shape,y.shape

((707, 96, 96, 1), (707,))

In [92]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=seed)

In [93]:
## Here is our model as a CNN
model = Sequential([
    Conv2D(32, (3,3), padding='same', input_shape=(ROW, COL, 1), activation='relu'),
    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dropout(.5),
    Dense(1, activation='softmax')
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 96, 96, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 24, 24, 64)       

In [94]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [95]:
## to save checkpoint to use later
modelpath = "model/phase.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [96]:
model.fit(x_train, y_train, batch_size=100, epochs=1000, validation_split=0.2,shuffle=True,
          callbacks=[checkpointer, early_stopping_callback])

Train on 452 samples, validate on 113 samples
Epoch 1/1000
452/452 [==============================] - 7s 16ms/step - loss: 0.6380 - accuracy: 0.6571 - val_loss: 0.4791 - val_accuracy: 0.8496

Epoch 00001: val_loss improved from inf to 0.47912, saving model to model/phase.hdf5
Epoch 2/1000
452/452 [==============================] - 7s 15ms/step - loss: 0.5328 - accuracy: 0.7965 - val_loss: 0.5297 - val_accuracy: 0.8496

Epoch 00002: val_loss did not improve from 0.47912
Epoch 3/1000
452/452 [==============================] - 7s 15ms/step - loss: 0.5572 - accuracy: 0.7965 - val_loss: 0.5224 - val_accuracy: 0.8496

Epoch 00003: val_loss did not improve from 0.47912
Epoch 4/1000
452/452 [==============================] - 7s 15ms/step - loss: 0.5215 - accuracy: 0.7965 - val_loss: 0.4705 - val_accuracy: 0.8496

Epoch 00004: val_loss improved from 0.47912 to 0.47045, saving model to model/phase.hdf5
Epoch 5/1000
452/452 [==============================] - 7s 16ms/step - loss: 0.5344 - accuracy

In [97]:
del model
model = load_model('model/phase.hdf5')

In [98]:
scores = model.evaluate(dog_x_test, dog_y_test, verbose=2)
print('MODEL ACCURACY: %.5f' % scores[1])

MODEL ACCURACY: 0.21429


In [99]:
ROW, COL = 96, 96
animals = []
aniaml = os.path.join(path,'*')
img = np.array(Image.open("../dataset/animal/bear.1.jpg"))
aniaml = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
aniaml = cv2.resize(aniaml, (ROW, COL))
aniaml = image.img_to_array(aniaml)
animals.append(aniaml)

animals = np.asarray(animals).astype('float32') / 255
classes = ['dog']
print(animals.shape)

(1, 96, 96, 1)


In [101]:
model.predict_classes(animals)[0][0]

1